# DataQuest Cleaning Data Mission Extension

This is an extension of the missions completed in [DataQuest](dataquest.io)'s Data Cleaning with Pandas.  I will use  [laptops.csv](https://dsserver-prod-resources-1.s3.amazonaws.com/293/laptops.csv?versionId=JU8WyvguPmkXgP_OjjIfZxACjL4hkAi2), a CSV file containing information about 1,300 laptop computers to explre the basics of data cleaning with pandas as we work.

- Convert the price_euros column to a numeric dtype.
- Extract the screen resolution from the screen column.
- Extract the processor speed from the cpu column.

Here are some questions you might like to answer in your own time by analyzing the cleaned data:
- Are laptops made by Apple more expensive than those made by other manufacturers?
- What is the best value laptop with a screen size of 15" or more?
- Which laptop has the most storage space?

In the first section of code I will load the data and explore its basic formatting.

## Load and Explore the data

In [1]:
# Import pandas
import pandas as pd
# Load the file
df = pd.read_csv('laptops.csv', encoding='Latin-1')
df.info()                 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
Manufacturer                1303 non-null object
Model Name                  1303 non-null object
Category                    1303 non-null object
Screen Size                 1303 non-null object
Screen                      1303 non-null object
CPU                         1303 non-null object
RAM                         1303 non-null object
 Storage                    1303 non-null object
GPU                         1303 non-null object
Operating System            1303 non-null object
Operating System Version    1133 non-null object
Weight                      1303 non-null object
Price (Euros)               1303 non-null object
dtypes: object(13)
memory usage: 132.4+ KB


Based on the above, we have 13 total columns with 1303 rows that contain the following information about the laptops:
* Manufacturer
* Model Name
* Category
* Screen Size
* Screen
* CPU
* RAM
* Storage
* GPU
* Operating System
* OPterating System Version
* Weight
* Price (Euros)

The data is in fair order, but we will need to make a number of corrections and adjustments to make it usable. In particular, we will want to:
1. Update the columns names to more conventional names
2. Change screen_size, ram, storage, weight, and price from objects into integers or floats
3. Explore the patterns beind the Null value sfor the Operating System Version

## Clean Data for laptops

### Correct column headers

One of the first steps I will take is to clean up the columns names by doing the following:
* Removing any leading or lagging spaces
* Removing "(" and "("
* Replacing spaces with"_" as is the convention
* Changing Upper case letters to lower case
* shorten Operating System to os

In [2]:
# write a function to corrects the columns
def col_correction(col):
    col = col.strip()
    col = col.replace('Operating System', 'os')
    col = col.replace('(', '')
    col = col.replace(')', '')
    col = col.replace(' ', '_')
    col = col.lower()
    return col

# Loop over the each column name and clean the columns
corr_col = []
for c in df.columns:
    clean_c = col_correction(c)
    corr_col.append(clean_c)

# replace columns of df and then check changes
df.columns = corr_col

In [3]:
print(df.dtypes)
df.head()

manufacturer    object
model_name      object
category        object
screen_size     object
screen          object
cpu             object
ram             object
storage         object
gpu             object
os              object
os_version      object
weight          object
price_euros     object
dtype: object


,manufacturer,model_name,category,screen_size,screen,cpu,ram,storage,gpu,os,os_version,weight,price_euros
0,Apple,MacBook Pro,Ultrabook,"13.3""",IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,NaN,1.37kg,"1339,69"
1,Apple,Macbook Air,Ultrabook,"13.3""",1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,NaN,1.34kg,"898,94"
2,HP,250 G6,Notebook,"15.6""",Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,NaN,1.86kg,"575,00"
3,Apple,MacBook Pro,Ultrabook,"15.4""",IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,NaN,1.83kg,"2537,45"
4,Apple,MacBook Pro,Ultrabook,"13.3""",IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,NaN,1.37kg,"1803,60"


### Explore the data for errors

Now, let's explore the data by examining the unique values for the relevant columns

In [4]:
explore_col = ['manufacturer', 'category', 'screen_size', 'screen', 
              'cpu', 'ram', 'storage', 'gpu', 'os', 'os_version', 'weight', 'price_euros']
for col in explore_col:
    print(df[col].value_counts())

Dell         297
Lenovo       297
HP           274
Asus         158
Acer         103
MSI           54
Toshiba       48
Apple         21
Samsung        9
Razer          7
Mediacom       7
Microsoft      6
Xiaomi         4
Vero           4
Google         3
Fujitsu        3
Chuwi          3
LG             3
Huawei         2
Name: manufacturer, dtype: int64
Notebook              727
Gaming                205
Ultrabook             196
2 in 1 Convertible    121
Workstation            29
Netbook                25
Name: category, dtype: int64
15.6"    665
14.0"    197
13.3"    164
17.3"    164
12.5"     39
11.6"     33
13.9"      6
12.0"      6
13.5"      6
12.3"      5
15.4"      4
10.1"      4
15.0"      4
13.0"      2
14.1"      1
17.0"      1
18.4"      1
11.3"      1
Name: screen_size, dtype: int64
Full HD 1920x1080                                507
1366x768                                         281
IPS Panel Full HD 1920x1080                      230
IPS Panel Full HD / Touchscreen 19

It looks like the *manufacturer*, *model_name*, *cpu*, *category*, *cpu*, *gpu*, and *os* are mostly correct except that I may want to extract additional information. 

I did find some errors and changes that should be made to the original data before I split any variables to extract relevant information.  These changes include the following:
* Remove """ from *screen_size* and convert to float
* Remove "GB" from *ram* and convert to integer and rename.
* Remove "kg" and "kgs" from *weight*, rename, convert to float
* Change "," to "." in price_euros and convert to float
* Correct the *os* columns. In particular, change "Mac OS" to "macOS"

### Correct errors and update dtypes

The columns are now more conventional, but we still need to change the types of each column into usable formats. Let's start with screen size. First, we will need to explore the data

#### Correct screen size

In [5]:
# Explore the unique vaules
df['screen_size'].unique()

array(['13.3"', '15.6"', '15.4"', '14.0"', '12.0"', '11.6"', '17.3"',
       '10.1"', '13.5"', '12.5"', '13.0"', '18.4"', '13.9"', '12.3"',
       '17.0"', '15.0"', '14.1"', '11.3"'], dtype=object)

In [6]:
# Remove '"' and convert to a float to preserve the decimal
df['screen_size'] = df['screen_size'].str.replace('"', '').astype(float)
# Rename column to clarify measurement
df.rename({'screen_size':'screen_size_inch'}, axis=1, inplace=True)

### ram

In [7]:
df['ram'].unique()

array(['8GB', '16GB', '4GB', '2GB', '12GB', '6GB', '32GB', '24GB', '64GB'],
      dtype=object)

In [8]:
# Remove 'GB' and convert to a float to preserve the decimal
df['ram'] = df['ram'].str.replace('GB', '').astype(int)
# Rename column to clarify measurement
df.rename({'ram':'ram_gb'}, axis=1, inplace=True)

### weight

In [9]:
df['weight'].unique()

array(['1.37kg', '1.34kg', '1.86kg', '1.83kg', '2.1kg', '2.04kg', '1.3kg',
       '1.6kg', '2.2kg', '0.92kg', '1.22kg', '0.98kg', '2.5kg', '1.62kg',
       '1.91kg', '2.3kg', '1.35kg', '1.88kg', '1.89kg', '1.65kg',
       '2.71kg', '1.2kg', '1.44kg', '2.8kg', '2kg', '2.65kg', '2.77kg',
       '3.2kg', '0.69kg', '1.49kg', '2.4kg', '2.13kg', '2.43kg', '1.7kg',
       '1.4kg', '1.8kg', '1.9kg', '3kg', '1.252kg', '2.7kg', '2.02kg',
       '1.63kg', '1.96kg', '1.21kg', '2.45kg', '1.25kg', '1.5kg',
       '2.62kg', '1.38kg', '1.58kg', '1.85kg', '1.23kg', '1.26kg',
       '2.16kg', '2.36kg', '2.05kg', '1.32kg', '1.75kg', '0.97kg',
       '2.9kg', '2.56kg', '1.48kg', '1.74kg', '1.1kg', '1.56kg', '2.03kg',
       '1.05kg', '4.4kg', '1.90kg', '1.29kg', '2.0kg', '1.95kg', '2.06kg',
       '1.12kg', '1.42kg', '3.49kg', '3.35kg', '2.23kg', '4.42kg',
       '2.69kg', '2.37kg', '4.7kg', '3.6kg', '2.08kg', '4.3kg', '1.68kg',
       '1.41kg', '4.14kg', '2.18kg', '2.24kg', '2.67kg', '2.14kg',
       '1.

In [10]:
# Remove 'kg' and 'kgs' and convert to a float to preserve the decimal
df['weight'] = df['weight'].str.replace('kg', '')
df['weight'] = df['weight'].str.replace('s', '').astype(float)
# Rename column to clarify measurement
df.rename({'weight':'weight_kg'}, axis=1, inplace=True)

### price_euros

In [11]:
df['price_euros'].unique()

array(['1339,69', '898,94', '575,00', '2537,45', '1803,60', '400,00',
       '2139,97', '1158,70', '1495,00', '770,00', '393,90', '344,99',
       '2439,97', '498,90', '1262,40', '1518,55', '745,00', '2858,00',
       '499,00', '979,00', '191,90', '999,00', '258,00', '819,00',
       '659,00', '418,64', '1099,00', '800,00', '1298,00', '896,00',
       '244,99', '199,00', '439,00', '1869,00', '998,00', '249,00',
       '367,00', '488,69', '879,00', '389,00', '1499,00', '522,99',
       '682,00', '1419,00', '369,00', '1299,00', '639,00', '466,00',
       '319,00', '841,00', '398,49', '1103,00', '384,00', '767,80',
       '586,19', '2449,00', '415,00', '599,00', '941,00', '690,00',
       '1983,00', '438,69', '229,00', '549,00', '949,00', '1089,00',
       '955,00', '870,00', '1095,00', '519,00', '855,00', '530,00',
       '977,00', '1096,16', '1510,00', '860,00', '399,00', '395,00',
       '1349,00', '699,00', '598,99', '1449,00', '1649,00', '689,00',
       '1197,00', '1195,00', '1049,0

Based on the above, I need to replace "," with "."

In [12]:
# Replace "," with "."  and convert to a float to preserve the decimal
df['price_euros'] = df['price_euros'].str.replace(',', '.').astype(float)

In [13]:
# Correct mappin of os
mapping_dict = {
    'Android': 'Android',
    'Chrome OS': 'Chrome OS',
    'Linux': 'Linux',
    'Mac OS': 'macOS',
    'No OS': 'No OS',
    'Windows': 'Windows',
    'macOS': 'macOS'
}

df['os'] = df['os'].map(mapping_dict)

In [14]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
manufacturer        1303 non-null object
model_name          1303 non-null object
category            1303 non-null object
screen_size_inch    1303 non-null float64
screen              1303 non-null object
cpu                 1303 non-null object
ram_gb              1303 non-null int32
storage             1303 non-null object
gpu                 1303 non-null object
os                  1303 non-null object
os_version          1133 non-null object
weight_kg           1303 non-null float64
price_euros         1303 non-null float64
dtypes: float64(3), int32(1), object(9)
memory usage: 127.3+ KB
None


,manufacturer,model_name,category,screen_size_inch,screen,cpu,ram_gb,storage,gpu,os,os_version,weight_kg,price_euros
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,NaN,1.37,1339.69
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,NaN,1.34,898.94
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,NaN,1.86,575.00
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,NaN,1.83,2537.45
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,NaN,1.37,1803.60


### Extract Additional Information from Columns

In this section, I will extract the extract the detailed information from some of the variables. 
* Split the storage column into 4 columns todal:
    * storage1_size: storage size for primary storage
    * storage1_type: storage type for primary storage
    * storage2_size: storage size for secondary storage
    * storage2_type: storage type for secondary storage
* Extract the screen resolution from the screen column and concatenate it as *screen_resoultion*
* Extract the cpu processor speed from the cpu column.

#### Storage

In [15]:
# Find unique values
df['storage'].value_counts()

256GB SSD                        412
1TB HDD                          224
500GB HDD                        132
512GB SSD                        118
128GB SSD +  1TB HDD              94
128GB SSD                         76
256GB SSD +  1TB HDD              73
32GB Flash Storage                38
2TB HDD                           16
64GB Flash Storage                15
512GB SSD +  1TB HDD              14
1TB SSD                           14
256GB SSD +  2TB HDD              10
1TB Hybrid                         9
256GB Flash Storage                8
16GB Flash Storage                 7
32GB SSD                           6
1GB SSD                            5
128GB Flash Storage                4
16GB SSD                           3
512GB SSD +  2TB HDD               3
256GB SSD +  500GB HDD             2
512GB Flash Storage                2
256GB SSD +  256GB SSD             2
128GB SSD +  2TB HDD               2
1TB SSD +  1TB HDD                 2
512GB SSD +  256GB SSD             1
2

The laptops in my list contain many different storage configurations.  Some laptops even come with two types of storage.  Therefore, I will first split the storage into two columns and then further divide the column into storage size and storage type. I then convert the size variables to integers.

In [16]:
# Split storage into two separate series
storage1 = df['storage'].str.split('+').str[0]
storage2 = df['storage'].str.split('+').str[1]
#Split each string again to extract store capacity, remove letters and convert to number
storage1_size_gb = storage1.str.split().str[0]
storage1_size_gb = storage1_size_gb.str.replace('GB', '')
storage1_size_gb = storage1_size_gb.str.replace('1TB', '1000')
storage1_size_gb = storage1_size_gb.str.replace('2TB', '2000')
storage1_size_gb = storage1_size_gb.astype(int)

storage1_type = storage1.str.split().str[1]

storage2_size_gb = storage2.str.split().str[0]
storage2_size_gb = storage2_size_gb.str.replace('GB', '')
storage2_size_gb = storage2_size_gb.str.replace('1TB', '1000')
storage2_size_gb = storage2_size_gb.str.replace('2TB', '2000')
storage2_size_gb.fillna(0, inplace=True)
storage2_size_gb = storage2_size_gb.astype(int)

storage2_type = storage2.str.split().str[1]
# Combine new columns and rename columns
storage_ext = pd.concat([storage1_size_gb, storage1_type, storage2_size_gb, storage2_type], axis=1)
storage_ext.columns = ['storage1_size', 'storage1_type', 'storage2_size', 'storage2_type']


# Add columns to the dataframe
df = pd.concat([df, storage_ext], axis=1).drop('storage',axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 16 columns):
manufacturer        1303 non-null object
model_name          1303 non-null object
category            1303 non-null object
screen_size_inch    1303 non-null float64
screen              1303 non-null object
cpu                 1303 non-null object
ram_gb              1303 non-null int32
gpu                 1303 non-null object
os                  1303 non-null object
os_version          1133 non-null object
weight_kg           1303 non-null float64
price_euros         1303 non-null float64
storage1_size       1303 non-null int32
storage1_type       1303 non-null object
storage2_size       1303 non-null int32
storage2_type       208 non-null object
dtypes: float64(3), int32(3), object(10)
memory usage: 147.7+ KB


#### Sreen Resolution

Extract the screen resolution from the screen column.

In [17]:
df['screen'].value_counts()

Full HD 1920x1080                                507
1366x768                                         281
IPS Panel Full HD 1920x1080                      230
IPS Panel Full HD / Touchscreen 1920x1080         53
Full HD / Touchscreen 1920x1080                   47
1600x900                                          23
Touchscreen 1366x768                              16
Quad HD+ / Touchscreen 3200x1800                  15
IPS Panel 4K Ultra HD 3840x2160                   12
IPS Panel 4K Ultra HD / Touchscreen 3840x2160     11
4K Ultra HD / Touchscreen 3840x2160               10
IPS Panel 1366x768                                 7
4K Ultra HD 3840x2160                              7
Touchscreen 2560x1440                              7
IPS Panel Retina Display 2560x1600                 6
Touchscreen 2256x1504                              6
IPS Panel Quad HD+ / Touchscreen 3200x1800         6
IPS Panel Retina Display 2304x1440                 6
IPS Panel Touchscreen 2560x1440               

After examining the unique values of the *screen* column, it is clear that the final digits contain the screen resolutions of the machines.  Therefore, I can extract these grabbing the last string after using a split function.

In [18]:
screen_temp = df['screen'].str.split(' ').str[-1]
screen_temp = pd.DataFrame(screen_temp)
screen_temp.columns = ['screen_resolution']
df = pd.concat([df, screen_temp], axis=1)

In [19]:
df.head()

,manufacturer,model_name,category,screen_size_inch,screen,cpu,ram_gb,gpu,os,os_version,weight_kg,price_euros,storage1_size,storage1_type,storage2_size,storage2_type,screen_resolution
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,Intel Iris Plus Graphics 640,macOS,NaN,1.37,1339.69,128,SSD,0,NaN,2560x1600
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,Intel HD Graphics 6000,macOS,NaN,1.34,898.94,128,Flash,0,NaN,1440x900
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,No OS,NaN,1.86,575.00,256,SSD,0,NaN,1920x1080
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,AMD Radeon Pro 455,macOS,NaN,1.83,2537.45,512,SSD,0,NaN,2880x1800
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,Intel Iris Plus Graphics 650,macOS,NaN,1.37,1803.60,256,SSD,0,NaN,2560x1600


#### Processor speed from cpu

In [20]:
df['cpu'].value_counts()

Intel Core i5 7200U 2.5GHz              190
Intel Core i7 7700HQ 2.8GHz             146
Intel Core i7 7500U 2.7GHz              134
Intel Core i7 8550U 1.8GHz               73
Intel Core i5 8250U 1.6GHz               72
Intel Core i5 6200U 2.3GHz               68
Intel Core i3 6006U 2GHz                 64
Intel Core i7 6500U 2.5GHz               49
Intel Core i7 6700HQ 2.6GHz              43
Intel Core i3 7100U 2.4GHz               37
Intel Core i5 7300HQ 2.5GHz              33
Intel Celeron Dual Core N3350 1.1GHz     30
Intel Celeron Dual Core N3060 1.6GHz     28
Intel Core i7 6600U 2.6GHz               18
Intel Core i3 6006U 2.0GHz               16
Intel Pentium Quad Core N4200 1.1GHz     14
Intel Core i5 7300U 2.6GHz               14
Intel Core i7 7600U 2.8GHz               13
AMD A9-Series 9420 3GHz                  11
Intel Core i5 6300U 2.4GHz               11
Intel Pentium Quad Core N3710 1.6GHz     11
Intel Celeron Dual Core N3050 1.6GHz     11
Intel Core i7 6820HK 2.7GHz     

It appears that the cpu processor speed is also at the end of the string, therefore, it can be extracted just like the screen resolution.

In [21]:
cpu_speed_temp = df['cpu'].str.split(' ').str[-1]
# Remove GHz
cpu_speed_temp = cpu_speed_temp.str.replace('GHz', '').astype(float)
cpu_speed_temp.columns = ['cpu_speed_ghz']
df = pd.concat([df, cpu_speed_temp], axis=1)
# Rename last column
df.head()

,manufacturer,model_name,category,screen_size_inch,screen,cpu,ram_gb,gpu,os,os_version,weight_kg,price_euros,storage1_size,storage1_type,storage2_size,storage2_type,screen_resolution,cpu
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,Intel Iris Plus Graphics 640,macOS,NaN,1.37,1339.69,128,SSD,0,NaN,2560x1600,2.3
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,Intel HD Graphics 6000,macOS,NaN,1.34,898.94,128,Flash,0,NaN,1440x900,1.8
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,No OS,NaN,1.86,575.00,256,SSD,0,NaN,1920x1080,2.5
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,AMD Radeon Pro 455,macOS,NaN,1.83,2537.45,512,SSD,0,NaN,2880x1800,2.7
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,Intel Iris Plus Graphics 650,macOS,NaN,1.37,1803.60,256,SSD,0,NaN,2560x1600,3.1


In [22]:
# Rename columns
col_names = ['manufacturer', 'model_name', 'category', 'screen_size_inch',
             'screen', 'cpu', 'ram_gb', 'gpu', 'os', 'os_version', 'weight_kg',
             'price_euros', 'storage1_size', 'storage1_type', 'storage2_size', 'storage2_type',
             'screen_resolution', 'cpu_speed_ghz']
df.columns = col_names

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 18 columns):
manufacturer         1303 non-null object
model_name           1303 non-null object
category             1303 non-null object
screen_size_inch     1303 non-null float64
screen               1303 non-null object
cpu                  1303 non-null object
ram_gb               1303 non-null int32
gpu                  1303 non-null object
os                   1303 non-null object
os_version           1133 non-null object
weight_kg            1303 non-null float64
price_euros          1303 non-null float64
storage1_size        1303 non-null int32
storage1_type        1303 non-null object
storage2_size        1303 non-null int32
storage2_type        208 non-null object
screen_resolution    1303 non-null object
cpu_speed_ghz        1303 non-null float64
dtypes: float64(4), int32(3), object(11)
memory usage: 168.0+ KB


### Correct null values

The only columns with null values is the os_version column. There are various ways to deal with missing values including:
* Remove any rows that have missing values.
* Remove any columns that have missing values.
* Fill the missing values with some other value.
* Leave the missing values as is.

Let's explore this one more to find out if there is a pattern to the null values

In [24]:
# Find the unique values for the null values in the os_version
print(df["os_version"].value_counts(dropna=False))

10      1072
NaN      170
7         45
X          8
10 S       8
Name: os_version, dtype: int64


In [25]:
df.loc[df["os_version"].isnull(),"os"].value_counts()

No OS        66
Linux        62
Chrome OS    27
macOS        13
Android       2
Name: os, dtype: int64

For the rows with missing *os_version*, the most frequent value is "No OS". This is important to note because if there is no os, there shouldn't be a version defined in the os_version column.

Thirteen of the laptops that come with macOS do not specify the version. We can use our knowledge of MacOS to confirm that os_version should be equal to X.

The rest of the missing versions will be left alone.

In [26]:
# Replace os_version with "Version Unkonwn when no OS is given
df.loc[df['os'] == 'No OS', 'os_version'] = 'Version Unknown'

# Replace macOS with missing version with "x"
df.loc[df["os"] == "macOS", "os_version"] = "X"
# Preview changes
df.loc[df['os_version'].isnull(), 'os'].value_counts()

Linux        62
Chrome OS    27
Android       2
Name: os, dtype: int64

In [27]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 18 columns):
manufacturer         1303 non-null object
model_name           1303 non-null object
category             1303 non-null object
screen_size_inch     1303 non-null float64
screen               1303 non-null object
cpu                  1303 non-null object
ram_gb               1303 non-null int32
gpu                  1303 non-null object
os                   1303 non-null object
os_version           1212 non-null object
weight_kg            1303 non-null float64
price_euros          1303 non-null float64
storage1_size        1303 non-null int32
storage1_type        1303 non-null object
storage2_size        1303 non-null int32
storage2_type        208 non-null object
screen_resolution    1303 non-null object
cpu_speed_ghz        1303 non-null float64
dtypes: float64(4), int32(3), object(11)
memory usage: 168.0+ KB
None


,manufacturer,model_name,category,screen_size_inch,screen,cpu,ram_gb,gpu,os,os_version,weight_kg,price_euros,storage1_size,storage1_type,storage2_size,storage2_type,screen_resolution,cpu_speed_ghz
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,Intel Iris Plus Graphics 640,macOS,X,1.37,1339.69,128,SSD,0,NaN,2560x1600,2.3
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,Intel HD Graphics 6000,macOS,X,1.34,898.94,128,Flash,0,NaN,1440x900,1.8
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,No OS,Version Unknown,1.86,575.00,256,SSD,0,NaN,1920x1080,2.5
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,AMD Radeon Pro 455,macOS,X,1.83,2537.45,512,SSD,0,NaN,2880x1800,2.7
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,Intel Iris Plus Graphics 650,macOS,X,1.37,1803.60,256,SSD,0,NaN,2560x1600,3.1


## Analysis

I will answer the three questions asked by the DataQuest team:
1. Are laptops made by Apple more expensive than those made by other manufacturers?
2. What is the best value laptop with a screen size of 15" or more?
3. Which laptop has the most storage space?

### Are Apple laptops more expensive?

In [28]:
# Isolate needed columns
df_manu_avg_price = df[['manufacturer', 'price_euros']]
# Calculat the mean laptop price for each manufacturer
manu_avg_price = df_manu_avg_price.groupby('manufacturer', as_index=False)['price_euros'].mean()
# Sort the dataframe by the price
manu_avg_price.sort_values(by='price_euros', inplace=True, ascending=False)
manu_avg_price

,manufacturer,price_euros
14,Razer,3346.142857
9,LG,2099.000000
11,MSI,1728.908148
6,Google,1677.666667
13,Microsoft,1612.308333
1,Apple,1564.198571
8,Huawei,1424.000000
15,Samsung,1413.444444
16,Toshiba,1267.812500
4,Dell,1186.068990


In [29]:
# Plot the average price in a horizontal bar graph.
import matplotlib.pyplot as plt

ax = manu_avg_price.plot.barh(x='manufacturer', y='price_euros', rot=0)
plt.show()

<Figure size 640x480 with 1 Axes>

Based on the analysis above, Apple does not have the most expensive laptop, on average.  Razer's average price dwarfs all others with an average price of $3,346 euros.  In fact Apple is number 6 on the list with Razer, LG, MSI, Google, and Microsoft with a higher average price.

### Best value laptop with screen size of 15" or more

To answer the larger question, I need to first explore two related questions:
1. What is considered a value laptop?
2. How can I find the best?

To answer these questions, I will first create a new dataframe that isolates laptops with 15in screens.

In [30]:
# Select only laptops with a screen size of greater than 15 inches
df_screen_15in = df.loc[df['screen_size_inch'] > 15, :]
df_screen_15in.head()

,manufacturer,model_name,category,screen_size_inch,screen,cpu,ram_gb,gpu,os,os_version,weight_kg,price_euros,storage1_size,storage1_type,storage2_size,storage2_type,screen_resolution,cpu_speed_ghz
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,No OS,Version Unknown,1.86,575.00,256,SSD,0,NaN,1920x1080,2.5
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,AMD Radeon Pro 455,macOS,X,1.83,2537.45,512,SSD,0,NaN,2880x1800,2.7
5,Acer,Aspire 3,Notebook,15.6,1366x768,AMD A9-Series 9420 3GHz,4,AMD Radeon R5,Windows,10,2.10,400.00,500,HDD,0,NaN,1366x768,3.0
6,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.2GHz,16,Intel Iris Pro Graphics,macOS,X,2.04,2139.97,256,Flash,0,NaN,2880x1800,2.2
10,HP,250 G6,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,4,Intel HD Graphics 620,No OS,Version Unknown,1.86,393.90,500,HDD,0,NaN,1366x768,2.5


Let's explore the laptops with 15 inch screens a little more in-depth.

In [31]:
df_screen_15in.describe()

,screen_size_inch,ram_gb,weight_kg,price_euros,storage1_size,storage2_size,cpu_speed_ghz
count,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000
mean,15.937964,8.742515,2.386913,1063.962754,524.435928,258.404790,2.382371
std,0.682556,5.625784,0.560184,735.863766,392.406511,475.970702,0.463905
min,15.400000,2.000000,1.080000,199.000000,8.000000,0.000000,1.100000
25%,15.600000,4.000000,2.060000,557.985000,256.000000,0.000000,2.000000
50%,15.600000,8.000000,2.200000,846.000000,500.000000,0.000000,2.500000
75%,15.600000,8.000000,2.500000,1299.000000,1000.000000,0.000000,2.700000
max,18.400000,64.000000,4.700000,6099.000000,2000.000000,2000.000000,3.600000


There are 835 laptops in our dataset with screen greater than 15 inches.  Their average price is 846 euros, and the 25th percentile is 557.985 euros.  Since we are considering value laptops, I will narrow our dataset further by exploring only the laptops that are below the 25 percentile price of 557.985 euros.

In [32]:
df_screen_15in_value = df_screen_15in.loc[df['price_euros'] < 557.985, :]
df_screen_15in_value.describe()

,screen_size_inch,ram_gb,weight_kg,price_euros,storage1_size,storage2_size,cpu_speed_ghz
count,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000
mean,15.704306,4.593301,2.150861,422.756077,688.861244,4.784689,2.042488
std,0.406475,1.411543,0.218637,82.779841,395.729444,69.171446,0.530012
min,15.600000,2.000000,1.680000,199.000000,16.000000,0.000000,1.100000
25%,15.600000,4.000000,2.000000,367.000000,500.000000,0.000000,1.600000
50%,15.600000,4.000000,2.180000,439.000000,500.000000,0.000000,2.000000
75%,15.600000,4.000000,2.200000,489.000000,1000.000000,0.000000,2.400000
max,17.300000,8.000000,2.800000,557.370000,2000.000000,1000.000000,3.600000


In [33]:
df_screen_15in_value.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209 entries, 5 to 1302
Data columns (total 18 columns):
manufacturer         209 non-null object
model_name           209 non-null object
category             209 non-null object
screen_size_inch     209 non-null float64
screen               209 non-null object
cpu                  209 non-null object
ram_gb               209 non-null int32
gpu                  209 non-null object
os                   209 non-null object
os_version           177 non-null object
weight_kg            209 non-null float64
price_euros          209 non-null float64
storage1_size        209 non-null int32
storage1_type        209 non-null object
storage2_size        209 non-null int32
storage2_type        1 non-null object
screen_resolution    209 non-null object
cpu_speed_ghz        209 non-null float64
dtypes: float64(4), int32(3), object(11)
memory usage: 28.6+ KB


I have now narrowed down the number of laptops to 209 by including only laptops with screen sizes greather than 15 inches and exlucing laptops prices at or above 557.985 euros.  I will consider these laptops as value laptops with screens greater than 15 inches.

Now, I need to determine which one is the best.  Two of the most important aspects of computer hardware is the processor speed and the amount of RAM. Let's explore both.

In [34]:
df_screen_15in_value['ram_gb'].value_counts()

4    165
8     26
6     14
2      4
Name: ram_gb, dtype: int64

In [35]:
df_screen_15in_value['cpu'].value_counts()

Intel Core i3 6006U 2GHz                 49
Intel Core i5 7200U 2.5GHz               21
Intel Core i3 7100U 2.4GHz               14
Intel Pentium Quad Core N4200 1.1GHz     12
Intel Celeron Dual Core N3350 1.1GHz     12
Intel Core i3 6006U 2.0GHz               11
AMD A9-Series 9420 3GHz                  11
Intel Celeron Dual Core N3060 1.6GHz      8
Intel Pentium Quad Core N3710 1.6GHz      7
Intel Celeron Dual Core N3050 1.6GHz      7
AMD A6-Series 9220 2.5GHz                 7
Intel Core i5 6200U 2.3GHz                7
AMD A8-Series 7410 2.2GHz                 4
AMD A9-Series 9410 2.9GHz                 3
Intel Celeron Dual Core 3205U 1.5GHz      3
AMD E-Series E2-9000e 1.5GHz              2
Intel Core i3 7130U 2.7GHz                2
AMD A6-Series A6-9220 2.5GHz              2
AMD A12-Series 9720P 3.6GHz               2
Intel Pentium Quad Core N3700 1.6GHz      2
Intel Celeron Dual Core N3350 2GHz        2
AMD E-Series 7110 1.8GHz                  2
Intel Core i3 6100U 2.3GHz      

In [36]:
df_screen_15in_value['cpu_speed_ghz'].value_counts()

2.00    63
2.50    30
1.60    28
1.10    24
2.40    15
3.00    11
2.30     9
2.20     8
1.50     6
2.90     5
2.70     3
3.60     2
1.80     2
1.44     2
2.10     1
Name: cpu_speed_ghz, dtype: int64

In [37]:
# Find the 75 percentile for processor speed.
df_screen_15in_value['cpu_speed_ghz'].quantile(q=0.75)
# pd.cut(cpu_speed_ghz, 4, retbins=True)

2.4

The majority of the computers have 4 GB of RAM followed by 8 GB. The 75th percentile for processor speed from the computers is 2.4 GHz.  Therefore, let's keep only laptops that have at elast 8 GB of RAM and a processor with a speed of at least 2.4 GHz.

Please note that processor speed is far from the only important aspect of a processor.  The number of cores, threads, amount of cache and the amount of energy required.  Please see [this overview]( https://www.tomshardware.com/reviews/cpu-buying-guide,5643.html) for more informaiton.

In [38]:
# Keep only laptops with 8 GB of RAM and a processor speed of at least 2.4 GHz.
df_screen_15in_value = df_screen_15in_value.loc[((df_screen_15in_value['ram_gb'] == 8) & (df_screen_15in_value['cpu_speed_ghz'] >= 2.4)),:]

In [39]:
df_screen_15in_value.describe()

,screen_size_inch,ram_gb,weight_kg,price_euros,storage1_size,storage2_size,cpu_speed_ghz
count,14.000000,14.0,14.000000,14.000000,14.000000,14.0,14.000000
mean,15.942857,8.0,2.322143,508.437143,1098.857143,0.0,2.592857
std,0.684683,0.0,0.250298,41.612927,566.846996,0.0,0.324545
min,15.600000,8.0,1.910000,429.000000,128.000000,0.0,2.400000
25%,15.600000,8.0,2.200000,476.507500,1000.000000,0.0,2.500000
50%,15.600000,8.0,2.250000,519.950000,1000.000000,0.0,2.500000
75%,15.600000,8.0,2.345000,539.712500,1000.000000,0.0,2.500000
max,17.300000,8.0,2.800000,557.370000,2000.000000,0.0,3.600000


In [40]:
df_screen_15in_value

,manufacturer,model_name,category,screen_size_inch,screen,cpu,ram_gb,gpu,os,os_version,weight_kg,price_euros,storage1_size,storage1_type,storage2_size,storage2_type,screen_resolution,cpu_speed_ghz
18,Lenovo,IdeaPad 320-15IKB,Notebook,15.6,Full HD 1920x1080,Intel Core i3 7100U 2.4GHz,8,Nvidia GeForce 940MX,No OS,Version Unknown,2.20,499.00,1000,HDD,0,NaN,1920x1080,2.4
42,Asus,X542UQ-GO005 (i5-7200U/8GB/1TB/GeForce,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,8,Nvidia GeForce 940MX,Linux,NaN,2.30,522.99,1000,HDD,0,NaN,1366x768,2.5
76,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,No OS,Version Unknown,2.20,519.00,2000,HDD,0,NaN,1920x1080,2.5
175,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i3 7100U 2.4GHz,8,Intel HD Graphics 620,Windows,10,2.30,459.00,1000,HDD,0,NaN,1366x768,2.4
213,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,No OS,Version Unknown,2.20,549.00,256,SSD,0,NaN,1920x1080,2.5
256,HP,15-BW094nd (A6-9220/8GB/128GB/W10),Notebook,15.6,1366x768,AMD A6-Series A6-9220 2.5GHz,8,AMD Radeon R4 Graphics,Windows,10,1.91,445.90,128,SSD,0,NaN,1366x768,2.5
303,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,8,Nvidia GeForce 940MX,No OS,Version Unknown,2.20,549.00,2000,HDD,0,NaN,1366x768,2.5
358,Dell,Inspiron 3567,Notebook,15.6,Touchscreen 1366x768,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,Windows,10,2.36,557.37,2000,HDD,0,NaN,1366x768,2.5
418,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,Windows,10,2.14,539.95,1000,HDD,0,NaN,1366x768,2.5
440,Lenovo,IdeaPad 320-15AST,Notebook,17.3,1600x900,AMD A6-Series 9220 2.5GHz,8,AMD Radeon R4,Windows,10,2.80,519.00,1000,HDD,0,NaN,1600x900,2.5


We are now down to 14 laptops. Let's keep only laptops with a screen Full HD screen(1920x1080 resolution)

In [41]:
df_screen_15in_value['screen_resolution'].value_counts()

1366x768     8
1600x900     3
1920x1080    3
Name: screen_resolution, dtype: int64

In [42]:
df_screen_15in_value = df_screen_15in_value.loc[(df_screen_15in_value['screen_resolution'] == '1920x1080'), :]

In [43]:
df_screen_15in_value

,manufacturer,model_name,category,screen_size_inch,screen,cpu,ram_gb,gpu,os,os_version,weight_kg,price_euros,storage1_size,storage1_type,storage2_size,storage2_type,screen_resolution,cpu_speed_ghz
18,Lenovo,IdeaPad 320-15IKB,Notebook,15.6,Full HD 1920x1080,Intel Core i3 7100U 2.4GHz,8,Nvidia GeForce 940MX,No OS,Version Unknown,2.2,499.0,1000,HDD,0,NaN,1920x1080,2.4
76,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,No OS,Version Unknown,2.2,519.0,2000,HDD,0,NaN,1920x1080,2.5
213,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,No OS,Version Unknown,2.2,549.0,256,SSD,0,NaN,1920x1080,2.5


We have now narrowed our list down to 3 laptops that are very similar as they are all Lenovo IdeaPad 320 models with small differences. I'll leave it at this for now. Let's summarize how we filtered the laptops
* Screen at least 15 inces.
* Price below the 25 percentile (557.985 euros) for our database
* 8 GB of RAM
* Processor speed of at least 2.4 GHz (7th percentile of remaining laptops)
* Full HD screen (1920x1080).

### Which laptop has the most storage space?

Finally, I will find the laptop with the most storage space by adding up the amount of storage in the primary (*storage1_size*) and secondary (*storage2_size*) hard drives.

In [44]:
# Calculate total storage for each laptop
df['total_storage'] = df['storage1_size'] + df['storage2_size']
# Find the max storeage space
df['total_storage'].max()

2512

The laptop with the most storage has 2512 GB. Let's find which laptop this is.

In [45]:
df.loc[df['total_storage'] == 2512, :]

,manufacturer,model_name,category,screen_size_inch,screen,cpu,ram_gb,gpu,os,os_version,weight_kg,price_euros,storage1_size,storage1_type,storage2_size,storage2_type,screen_resolution,cpu_speed_ghz,total_storage
370,Asus,ZenBook Flip,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,12,Nvidia GeForce GT 940MX,Windows,10,2.26,1099.0,512,SSD,2000,HDD,1920x1080,2.7,2512
894,MSI,GS73VR Stealth,Gaming,17.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 6700HQ 2.6GHz,16,Nvidia GeForce GTX 1060,Windows,10,2.43,2649.0,512,SSD,2000,HDD,3840x2160,2.6,2512
977,Asus,Q534UX-BHI7T19 (i7-7500U/16GB/2TB,2 in 1 Convertible,15.6,4K Ultra HD / Touchscreen 3840x2160,Intel Core i7 7500U 2.7GHz,16,Nvidia GeForce GTX 950M,Windows,10,2.30,1799.0,512,SSD,2000,HDD,3840x2160,2.7,2512


In fact, there are three laptops with a storage capacity of 2,512 GB. Each one comes with a 512 GB SSD as the primary drive with a 2TB HDD as the secondary drive.